In [2]:
import pandas as pd

In [3]:
path = '/Users/idris/Documents/ds_project/forecast_store_sales_kaggle/'

In [4]:
df = pd.read_csv(path + '/data/train.csv')
df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


 ##EDA

In [ ]:
df.isnull().sum()

In [ ]:
df.id.nunique(), df.groupby('id').size().max()

In [ ]:
df.family.nunique()

In [ ]:
df.store_nbr.nunique()

In [ ]:
df.groupby('store_nbr')['family'].nunique()

In [ ]:
df.sales.max(), df.sales.min(), df.sales.mean()

In [ ]:
df.date.min(), df.date.max()

In [ ]:
pd.date_range(df.date.min(), df.date.max()).shape

In [ ]:
df.groupby(['store_nbr', 'family']).size().max(), df.groupby(['store_nbr', 'family']).size().min()

In [ ]:
df[df.family=='BABY CARE'].shape

In [ ]:
df[(df.family=='BABY CARE') & (df.store_nbr == 1)].shape

In [ ]:
df[(df.family=='BABY CARE') & (df.store_nbr == 1)].head()

In [ ]:
df.dtypes

#data clean, pas de na, manque 3 dates pour chaue time series, 
#formatage de la data, index date, bu, fam, sales

In [17]:
df.date = pd.to_datetime(df.date)
dfts = df.set_index('date')#.drop('id',axis=1)
dfts.columns= ['id', 'bu', 'fam', 'sales', 'onprom']
dfts['famid'] = pd.factorize(dfts['fam'])[0]
dfts['ts_id'] = dfts['bu'].astype('str') + '_' + dfts['famid'].astype('str')

In [18]:
dfdate = pd.DataFrame({'date': pd.date_range(df.date.min(), df.date.max())})


In [19]:
dff = dfts[['ts_id']].drop_duplicates().merge(dfdate, how='cross')
dff = dff.merge(dfts.reset_index()[['id','date', 'ts_id', 'bu', 'famid', 'sales', 'onprom']], on=['date', 'ts_id'],
                  how='left')

In [20]:
dfts[['ts_id']].drop_duplicates().shape, 1688 * 1782, dff.shape

((1782, 1), 3008016, (3008016, 7))

In [21]:
dff.head()

,ts_id,date,id,bu,famid,sales,onprom
0,1_0,2013-01-01,0.0,1.0,0.0,0.0,0.0
1,1_0,2013-01-02,1782.0,1.0,0.0,2.0,0.0
2,1_0,2013-01-03,3564.0,1.0,0.0,3.0,0.0
3,1_0,2013-01-04,5346.0,1.0,0.0,3.0,0.0
4,1_0,2013-01-05,7128.0,1.0,0.0,5.0,0.0


In [22]:
dff.isnull().sum()

ts_id        0
date         0
id        7128
bu        7128
famid     7128
sales     7128
onprom    7128
dtype: int64

In [23]:
dff['bu'] = dff['ts_id'].apply(lambda x: int(x.split('_')[0]))
dff['famid'] = dff['ts_id'].apply(lambda x: int(x.split('_')[1]))
dff['sales'] = dff['sales'].fillna(0)
dff['onprom'] = dff['onprom'].fillna(method='bfill')
dff['id'] = dff['id'].fillna(-1)

In [24]:
dff.head()

,ts_id,date,id,bu,famid,sales,onprom
0,1_0,2013-01-01,0.0,1,0,0.0,0.0
1,1_0,2013-01-02,1782.0,1,0,2.0,0.0
2,1_0,2013-01-03,3564.0,1,0,3.0,0.0
3,1_0,2013-01-04,5346.0,1,0,3.0,0.0
4,1_0,2013-01-05,7128.0,1,0,5.0,0.0


In [25]:
dffirst = dff[dff.sales > 0].groupby('ts_id')['date'].first().reset_index()
dffirst.columns = ['ts_id', 'firstdate']

In [26]:
dff = dff.merge(dffirst, on=['ts_id'], how='left')

In [27]:
dff.head()

,ts_id,date,id,bu,famid,sales,onprom,firstdate
0,1_0,2013-01-01,0.0,1,0,0.0,0.0,2013-01-02
1,1_0,2013-01-02,1782.0,1,0,2.0,0.0,2013-01-02
2,1_0,2013-01-03,3564.0,1,0,3.0,0.0,2013-01-02
3,1_0,2013-01-04,5346.0,1,0,3.0,0.0,2013-01-02
4,1_0,2013-01-05,7128.0,1,0,5.0,0.0,2013-01-02


In [28]:
dff.firstdate = dff.firstdate.fillna(pd.to_datetime('2033-12-31'))

In [29]:
dff.ts_id.nunique()

1782

In [30]:
dff = dff[dff['date'] >= dff['firstdate']]

In [31]:
dff.ts_id.nunique()

1729

In [41]:
dff[dff.ts_id == '1_4']

,ts_id,date,id,bu,famid,sales,onprom
8133,1_4,2016-10-13,2455600,1,4,2.0,0.0
8134,1_4,2016-10-14,2457382,1,4,2.0,0.0
8135,1_4,2016-10-15,2459164,1,4,3.0,0.0
8136,1_4,2016-10-16,2460946,1,4,2.0,0.0
8137,1_4,2016-10-17,2462728,1,4,2.0,0.0
...,...,...,...,...,...,...,...
8435,1_4,2017-08-11,2991982,1,4,0.0,0.0
8436,1_4,2017-08-12,2993764,1,4,0.0,0.0
8437,1_4,2017-08-13,2995546,1,4,0.0,0.0
8438,1_4,2017-08-14,2997328,1,4,0.0,0.0


In [38]:
dff = dff.drop('firstdate', axis=1)
dff['id'] = dff['id'].astype(int)

In [40]:
dff.to_csv('data/trainclean.csv', index=False, sep=';')

In [39]:
dff.dtypes

ts_id             object
date      datetime64[ns]
id                 int64
bu                 int64
famid              int64
sales            float64
onprom           float64
dtype: object

In [42]:
dff.columns

Index(['ts_id', 'date', 'id', 'bu', 'famid', 'sales', 'onprom'], dtype='object')